In [2]:
import os
import pandas as pd
import numpy as np
from numpy import ma
from scipy import stats, linalg
from six import callable, string_types
import matplotlib.pyplot as plt

from TransferEntropy import TransferEntropy, LaggedTimeSeries, AutoBins

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
test_data = pd.read_csv('./Testing/Test_Utils/test_data.csv')
test_numpy = test_data.to_numpy()[:,1:]

In [3]:
test_data

,date,S1,S2,S3,S4
0,01/01/2018,22,-0.054181,0.306740,5.177009
1,02/01/2018,35,-0.038264,0.423752,6.641647
2,03/01/2018,64,0.005967,0.451472,8.252325
3,04/01/2018,40,-0.002890,0.470023,6.488850
4,05/01/2018,6,0.004926,0.309367,3.046403
...,...,...,...,...,...
535,20/06/2019,10,0.002566,0.446053,3.931973
536,21/06/2019,44,-0.001896,0.450318,7.164833
537,22/06/2019,9,-0.009153,0.329843,3.325823
538,23/06/2019,71,-0.003754,0.320296,8.998681


In [4]:
N = complex(20)
[slice(dim.min(),dim.max(),N) for dimname, dim in test_data.iteritems()]

/var/folders/5h/9cd9pzvx7rx5k8ty8zvbqc380000gn/T/ipykernel_3995/3265419399.py:2: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  [slice(dim.min(),dim.max(),N) for dimname, dim in test_data.iteritems()]


[slice('01/01/2018', '31/12/2018', (20+0j)),
 slice(0, 293, (20+0j)),
 slice(-0.212948, 0.215818, (20+0j)),
 slice(0.255755181, 0.482722312, (20+0j)),
 slice(0.013185229, 17.89226835, (20+0j))]

In [5]:
test_numpy

array([[22, -0.054181, 0.306739592, 5.177008664],
       [35, -0.038264, 0.423752082, 6.641647156],
       [64, 0.005967, 0.451471786, 8.252324509],
       ...,
       [9, -0.009153, 0.329843413, 3.325822994],
       [71, -0.003754, 0.320295918, 8.99868101],
       [41, -0.137054, 0.290409148, 7.063219041]], dtype=object)

In [6]:
N = complex(20)
[slice(dim.min(),dim.max(),N) for dim in test_numpy.T]

[slice(0, 293, (20+0j)),
 slice(-0.212948, 0.215818, (20+0j)),
 slice(0.255755181, 0.482722312, (20+0j)),
 slice(0.013185229, 17.89226835, (20+0j))]

In [7]:
TE = TransferEntropy(   DF = test_data,
                        endog = 'S2',     # Dependent Variable
                        exog = 'S3',      # Independent Variable
                        lag = 2
                        )

In [8]:
TE.lts.df

,date,S1,S2,S3,S4,date_lag2,S1_lag2,S2_lag2,S3_lag2,S4_lag2
2,03/01/2018,64,0.005967,0.451472,8.252325,01/01/2018,22.0,-0.054181,0.306740,5.177009
3,04/01/2018,40,-0.002890,0.470023,6.488850,02/01/2018,35.0,-0.038264,0.423752,6.641647
4,05/01/2018,6,0.004926,0.309367,3.046403,03/01/2018,64.0,0.005967,0.451472,8.252325
5,06/01/2018,5,0.035660,0.338410,2.878061,04/01/2018,40.0,-0.002890,0.470023,6.488850
6,07/01/2018,34,-0.014597,0.388144,6.195224,05/01/2018,6.0,0.004926,0.309367,3.046403
...,...,...,...,...,...,...,...,...,...,...
535,20/06/2019,10,0.002566,0.446053,3.931973,18/06/2019,5.0,0.003326,0.339382,3.029323
536,21/06/2019,44,-0.001896,0.450318,7.164833,19/06/2019,5.0,0.080539,0.431670,3.012577
537,22/06/2019,9,-0.009153,0.329843,3.325823,20/06/2019,10.0,0.002566,0.446053,3.931973
538,23/06/2019,71,-0.003754,0.320296,8.998681,21/06/2019,44.0,-0.001896,0.450318,7.164833


In [66]:
%timeit TE.nonlinear_TE(pdf_estimator = 'kernel', n_shuffles=10)

1.23 s ± 35.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
# skip window strolling

class LaggedTimeSeriesDF():
    def __init__(self, df, lag=None):
        self.df = df

        if lag is not None:
            self.t = lag
            self.df = self.__apply_lags__()
    
    def __apply_lags__(self):
        new_df = self.df.copy(deep=True).dropna()
        col_names = self.df.columns.values.tolist()
        # generate lagged array with t lags
        for col_name in col_names:
                new_df[col_name + '_lag' + str(self.t)] = self.df[col_name].shift(self.t)

        return new_df.iloc[self.t:]

class GrangerCausality():
    def __init__(self, lts_array, endog, exog, lag = None):
        ''' input LaggedTimeSeriesDF object as lts_df'''
        self.array = lts_array
        self.endog = endog  # Dependent Variable Y
        self.exog = exog  # Independent Variable X
        self.lag = lag

    def Granger_Caus(self, df=None, n_shuffles=0):
        ## Prepare lists for storing results
        granger_causalities = [0,0]
        GCs = []
        shuffled_TEs = []
        p_values = []
        z_scores = []

        df = np.copy(self.array)

        ## Require us to compare information transfer bidirectionally
        for i in range(2):
            ## Calculate Residuals after OLS Fitting, for both Independent and Joint Cases
            joint_residuals = self.ols_res_cal(df[:, i], df[:, [i+2, 3-i]])
            independent_residuals = self.ols_res_cal(df[:, i], df[:, i+2].reshape(-1, 1))

            ## Use Geweke's formula for Granger Causality 
            granger_causalities[i] = self.granger_causality(independent_residuals, joint_residuals)

        GCs.append(granger_causalities)
        ## Calculate Significance of GC during this window
        if n_shuffles > 0:
            p, z, TE_mean = significance(df=df,
                                            TE=granger_causalities,
                                            endog=self.endog,
                                            exog=self.exog,
                                            lag=self.lag,
                                            n_shuffles=n_shuffles,
                                            method='granger_causality')

            shuffled_TEs.append(TE_mean)
            p_values.append(p)
            z_scores.append(z)
            # column = [XY, YX]
            # rows = [TE, p_value, z_score, shuffled_TE]
            self.results = np.concatenate(
                (np.array(GCs), np.array(p_values), np.array(z_scores), np.array(shuffled_TEs)), axis=0)
        else:
            ## Store Granger Causality from X(t)->Y(t) and from Y(t)->X(t)
            self.results = np.array(GCs)

        return self.results
    
    #@njit
    def ols_res_cal(self, y, x):
        x = np.append(np.ones((len(x), 1)), x, axis=1)
        ins = np.linalg.inv(np.dot(x.T, x))
        out = np.dot(x.T, y)
        beta = np.dot(ins, out)
        res = y - np.dot(x, beta)
        return res
    
    #@njit
    def granger_causality(self, independent_residuals, joint_residuals):
        ind_res_var = np.var(independent_residuals) + np.finfo(float).eps
        jnt_res_var = np.var(joint_residuals) + np.finfo(float).eps
        gc =  np.log(ind_res_var / jnt_res_var)
        return gc

In [10]:
lts_df = LaggedTimeSeriesDF(test_data[['S2', 'S3']], lag=2).df.to_numpy()
GrangerCausality(lts_df, 'S2', 'S1', lag=2).Granger_Caus(n_shuffles=20).transpose().tolist()

[[0.0042882572997716755, 0.2, 0.6763686505785762, 0.0022425761872889413],
 [0.001450396826136952, 0.45, 0.04280969182065394, 0.0013972101907776286]]

In [23]:
lts_df.df

,S2,S3,S2_lag2,S3_lag2
2,0.005967,0.451472,-0.054181,0.306740
3,-0.002890,0.470023,-0.038264,0.423752
4,0.004926,0.309367,0.005967,0.451472
5,0.035660,0.338410,-0.002890,0.470023
6,-0.014597,0.388144,0.004926,0.309367
...,...,...,...,...
535,0.002566,0.446053,0.003326,0.339382
536,-0.001896,0.450318,0.080539,0.431670
537,-0.009153,0.329843,0.002566,0.446053
538,-0.003754,0.320296,-0.001896,0.450318


In [59]:
class TransferEntropyArray():
    def __init__(self, lts_array, endog, exog, lag = None, method=None):
        ''' input LaggedTimeSeriesDF object to numpy array lts_array'''
        self.array = lts_array
        self.endog = endog  # Dependent Variable Y
        self.exog = exog  # Independent Variable X
        self.lag = lag

        self.covars = self.covariance_cal(method)

    # @njit
    def covariance_cal(self, method):
        self.covars = [[], []]
        for i in range(2):

            if method == 'mi':
                self.covars[i] = [np.ones(shape=(1, 1)) * np.var(self.array[:, [1 - i]]),
                                  np.ones(shape=(1, 1)) * np.var(self.array[:, [i]]),
                                  np.cov(self.array[:, [1 - i, i]].T)]
            else:
                self.covars[i] = [np.cov(self.array[:, [i, i+2, 3-i]].T),
                                  np.cov(self.array[:, [3-i, i+2]].T),
                                  np.cov(self.array[:, [i, i+2]].T),
                                  np.ones(shape=(1, 1)) * np.var(self.array[:, [i+2]])]
        return self.covars
    
    def TransferEntropy(self, df=None, bandwidth=None, gridpoints=20, n_shuffles=0):
        ## Prepare lists for storing results
        TEs = []
        shuffled_TEs = []
        p_values = []
        z_scores = []

        df = np.copy(self.array)

        ## Initialise list to return TEs
        transfer_entropies = [0, 0]

        ## Require us to compare information transfer bidirectionally
        for i in range(2):
            ### Entropy calculated using Probability Density Estimation:
            ### Estimate PDF using Gaussian Kernels and use H(x) = p(x) log p(x)
            ## 1. H(Y,Y-t,X-t)
            H1 = self.get_entropy(df=df[:, [i, i+2, 3-i]],
                                gridpoints=gridpoints,
                                bandwidth=bandwidth,
                                covar=self.covars[i][0])
            ## 2. H(Y-t,X-t)
            H2 = self.get_entropy(df=df[:, [3-i, i+2]],
                                gridpoints=gridpoints,
                                bandwidth=bandwidth,
                                covar=self.covars[i][1])
            ## 3. H(Y,Y-t)
            H3 = self.get_entropy(df=df[:, [i, i+2]],
                                gridpoints=gridpoints,
                                bandwidth=bandwidth,
                                covar=self.covars[i][2])
            ## 4. H(Y-t)
            H4 = self.get_entropy(df=df[:, [i+2]],
                                gridpoints=gridpoints,
                                bandwidth=bandwidth,
                                covar=self.covars[i][3])

            ### Calculate Conditonal Entropy using: H(Y|X-t,Y-t) = H(Y,X-t,Y-t) - H(X-t,Y-t)
            conditional_entropy_joint = H1 - H2

            ### And Conditional Entropy independent of X(t) H(Y|Y-t) = H(Y,Y-t) - H(Y-t)            
            conditional_entropy_independent = H3 - H4

            ### Directional Transfer Entropy is the difference between the conditional entropies
            transfer_entropies[i] = conditional_entropy_independent - conditional_entropy_joint

        TEs.append(transfer_entropies)

        ## Calculate Significance of TE during this window
        if n_shuffles > 0:
            p, z, TE_mean = significance(df=df,
                                            TE=transfer_entropies,
                                            endog=self.endog,
                                            exog=self.exog,
                                            lag=self.lag,
                                            n_shuffles=n_shuffles,
                                            bandwidth=bandwidth,
                                            method='transfer_entropy')

            shuffled_TEs.append(TE_mean)
            p_values.append(p)
            z_scores.append(z)
            ## Store Significance Transfer Entropy from X(t)->Y(t) and from Y(t)->X(t)
            self.results = np.concatenate((np.array(TEs), np.array(p_values), np.array(z_scores), np.array(shuffled_TEs)), axis=0)
        else:
            ## Store Significance Transfer Entropy from X(t)->Y(t) and from Y(t)->X(t)
            self.results = np.array(TEs)
            
        return self.results
    
    def MutualInformation(self, df=None, bandwidth=None, gridpoints=20, n_shuffles=0):
        ## Prepare lists for storing results
        Norm_MIs = []
        shuffled_MIs = []
        p_values = []
        z_scores = []

        df = np.copy(self.array)

        ## Initialise list to return TEs
        # mutual_informations = [0, 0]
        normalized_mi = [0, 0]

        ## Require us to compare information transfer bidirectionally
        for i in range(2):
            ### Entropy calculated using Probability Density Estimation:
            ### Estimate PDF using Gaussian Kernels and use H(x) = p(x) log p(x)
            ## 1. H(Y,Y-t,X-t)
            H1 = self.get_entropy(df=df[:, [1 - i]],
                                gridpoints=gridpoints,
                                bandwidth=bandwidth,
                                covar=self.covars[i][0])
            ## 2. H(Y-t,X-t)
            H2 = self.get_entropy(df=df[:, [i]],
                                gridpoints=gridpoints,
                                bandwidth=bandwidth,
                                covar=self.covars[i][1])
            ## 3. H(Y,Y-t)
            H3 = self.get_entropy(df=df[:, [1 - i, i]],
                                gridpoints=gridpoints,
                                bandwidth=bandwidth,
                                covar=self.covars[i][2])

            normalized_mi[i] = 1 - (H1 + H2 - H3 / max(H1, H2))

        Norm_MIs.append(normalized_mi)

        ## Calculate Significance of TE during this window
        if n_shuffles > 0:
            p, z, MI_mean = significance(df=df,
                                            TE=normalized_mi,
                                            endog=self.endog,
                                            exog=self.exog,
                                            lag=self.lag,
                                            n_shuffles=n_shuffles,
                                            bandwidth=bandwidth,
                                            method='mutual_information')

            shuffled_MIs.append(MI_mean)
            p_values.append(p)
            z_scores.append(z)
            ## Store Significance Transfer Entropy from X(t)->Y(t) and from Y(t)->X(t)
            self.results = np.concatenate((np.array(Norm_MIs), np.array(p_values), np.array(z_scores), np.array(shuffled_MIs)), axis=0)
        else:
            ## Store Significance Transfer Entropy from X(t)->Y(t) and from Y(t)->X(t)
            self.results = np.array(Norm_MIs)
            
        return self.results

    def get_entropy(self, df, gridpoints=20, bandwidth=None, covar=None):
        """
            Function for calculating entropy from a probability mass 
            
        Args:
            df          -       (DataFrame) Samples over which to estimate density
            gridpoints  -       (int)       Number of gridpoints when integrating KDE over 
                                            the domain. Used if estimator='kernel'
            bandwidth   -       (float)     Bandwidth for KDE (scalar multiple to covariance
                                            matrix). Used if estimator='kernel'
            estimator   -       (string)    'histogram' or 'kernel'
            bins        -       (Dict of lists) Bin edges for NDHistogram. Used if estimator
                                            = 'histogram'
            covar       -       (Numpy ndarray) Covariance matrix between dimensions of df. 
                                            Used if estimator = 'kernel'
        Returns:
            entropy     -       (float)     Shannon entropy in bits

        """
        # df is np.array
        pdf = self.pdf_kde(df, gridpoints, bandwidth, covar)
        # log base 2 returns H(X) in bits
        return -np.sum(pdf * ma.log2(pdf).filled(0))
    
    def pdf_kde(self, df, gridpoints=None, bandwidth=1, covar=None):
        """
            Function for non-parametric density estimation using Kernel Density Estimation

        Args:
            df          -       (DataFrame) Samples over which to estimate density
            gridpoints  -       (int)       Number of gridpoints when integrating KDE over 
                                            the domain. Used if estimator='kernel'
            bandwidth   -       (float)     Bandwidth for KDE (scalar multiple to covariance
                                            matrix).
            covar       -       (Numpy ndarray) Covariance matrix between dimensions of df. 
                                            If None, these are calculated from df during the 
                                            KDE analysis

        Returns:
            Z/Z.sum()   -       (Numpy ndarray) Probability of a sample being between
                                            specific gridpoints (technically a probability mass)
        """
        # df is np.array
        ## Create Meshgrid to capture data
        if gridpoints is None:
            gridpoints = 20
        
        N = complex(gridpoints)
        
        slices = [slice(dim.min(),dim.max(),N) for dim in df.T]
        grids = np.mgrid[slices]

        ## Pass Meshgrid to Scipy Gaussian KDE to Estimate PDF
        positions = np.vstack([X.ravel() for X in grids])
        values = df.T
        kernel = _kde_(values, bw_method=bandwidth, covar=covar)
        Z = np.reshape(kernel(positions).T, grids[0].shape) 

        ## Normalise 
        return Z/Z.sum()


class _kde_(stats.gaussian_kde):
    """
    Subclass of scipy.stats.gaussian_kde. This is to enable the passage of a pre-defined covariance matrix, via the
    `covar` parameter. This is handled internally within TransferEntropy class.
    The matrix is calculated on the overall dataset, before windowing, which allows for consistency between windows,
    and avoiding duplicative computational operations, compared with calculating the covariance each window.

    Functions left as much as possible identical to scipi.stats.gaussian_kde; docs available:
    https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gaussian_kde.html
    """
    def __init__(self, dataset, bw_method=None, df=None, covar=None):
        self.dataset = np.atleast_2d(dataset)
        if not self.dataset.size > 1:
            raise ValueError("`dataset` input should have multiple elements.")

        self.d, self.n = self.dataset.shape
        self.set_bandwidth(bw_method=bw_method, covar=covar)


    def set_bandwidth(self, bw_method=None, covar=None):
        
        if bw_method is None:
            pass
        elif np.isscalar(bw_method) and not isinstance(bw_method, string_types):
            self._bw_method = 'use constant'
            self.covariance_factor = lambda: bw_method
        else:
            msg = "`bw_method` should be 'scott', 'silverman', a scalar " \
                  "or a callable."
            raise ValueError(msg)

        self._compute_covariance(covar)

    def _compute_covariance(self, covar):
        """Computes the covariance matrix for each Gaussian kernel using
        covariance_factor().
        """
        if covar is not None:
            self._data_covariance = covar
            self._data_inv_cov = linalg.inv(self._data_covariance)
        
        self.factor = self.covariance_factor()
        # Cache covariance and Cholesky decomp of covariance
        if not hasattr(self, '_data_cho_cov'):
            self._data_covariance = np.atleast_2d(np.cov(self.dataset, rowvar=1,
                                            bias=False))
            self._data_cho_cov = linalg.cholesky(self._data_covariance,
                                                lower=True)

        self.covariance = self._data_covariance * self.factor**2
        self.cho_cov = (self._data_cho_cov * self.factor).astype(np.float64)
        self.log_det = 2*np.log(np.diag(self.cho_cov
                                        * np.sqrt(2*np.pi))).sum()

    @property
    def inv_cov(self):
        # Re-compute from scratch each time because I'm not sure how this is
        # used in the wild. (Perhaps users change the `dataset`, since it's
        # not a private attribute?) `_compute_covariance` used to recalculate
        # all these, so we'll recalculate everything now that this is a
        # a property.
        self.factor = self.covariance_factor()
        self._data_covariance = np.atleast_2d(np.cov(self.dataset, rowvar=1,
                                        bias=False, aweights=self.weights))
        return linalg.inv(self._data_covariance) / self.factor**2

In [64]:
%timeit TransferEntropyArray(lts_df, 'S2', 'S1', lag=2).TransferEntropy(n_shuffles=10)

1.16 s ± 27.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
def significance(df, TE, endog, exog, lag, n_shuffles, method, bandwidth=None):
    """
        Perform significance analysis on the hypothesis test of statistical causality, for both X(t)->Y(t)
        and Y(t)->X(t) directions
   
        Calculated using:  Assuming stationarity, we shuffle the time series to provide the null hypothesis. 
                           The proportion of tests where TE > TE_shuffled gives the p-value significance level.
                           The amount by which the calculated TE is greater than the average shuffled TE, divided
                           by the standard deviation of the results, is the z-score significance level.

        Arguments:
            TE              -      (list)    Contains the transfer entropy in each direction, i.e. [TE_XY, TE_YX]
            endog           -      (string)  The endogenous variable in the TE analysis being significance tested (i.e. X or Y) 
            exog            -      (string)  The exogenous variable in the TE analysis being significance tested (i.e. X or Y) 
            pdf_estimator   -      (string)  The pdf_estimator used in the original TE analysis
            bins            -      (Dict of lists)  The bins used in the original TE analysis

            n_shuffles      -      (float) Number of times to shuffle the dataframe, destroyig temporality
            both            -      (Bool) Whether to shuffle both endog and exog variables (z-score) or just exog                                  variables (giving z*-score)  
        Returns:
            p_value         -      Probablity of observing the result given the null hypothesis
            z_score         -      Number of Standard Deviations result is from mean (normalised)
        """

    ## Prepare array for Transfer Entropy of each Shuffle
    shuffled_TEs = np.zeros(shape=(2, n_shuffles))

    for i in range(n_shuffles):
        ## Perform Shuffle
        df = shuffle_along_axis(df, axis=0)

        if method == 'granger_causality':
            ## Calculate New TE
            shuffled_causality = GrangerCausality(df, endog=endog, exog=exog, lag=lag)
            TE_shuffled = shuffled_causality.Granger_Caus(df, n_shuffles=0)
        elif method == 'mutual information':
            ## Calculate New TE
            shuffled_causality = TransferEntropyArray(df, endog=endog, exog=exog, lag=lag, method = 'mi')
            TE_shuffled = shuffled_causality.MutualInformation(df, bandwidth, n_shuffles=0)
        else:
            ## Calculate New TE
            shuffled_causality = TransferEntropyArray(df, endog=endog, exog=exog, lag=lag)
            TE_shuffled = shuffled_causality.TransferEntropy(df, bandwidth, n_shuffles=0)
        shuffled_TEs[:, i] = TE_shuffled

    ## Calculate p-values for each direction
    p_values = (np.count_nonzero(TE[0] < shuffled_TEs[0, :]) / n_shuffles, \
                np.count_nonzero(TE[1] < shuffled_TEs[1, :]) / n_shuffles)

    shuff_te_zero = np.std(shuffled_TEs[0, :]) + np.finfo(float).eps
    shuff_te_one = np.std(shuffled_TEs[1, :]) + np.finfo(float).eps

    ## Calculate z-scores for each direction
    z_scores = ((TE[0] - np.mean(shuffled_TEs[0, :])) / shuff_te_zero, \
                (TE[1] - np.mean(shuffled_TEs[1, :])) / shuff_te_one)

    TE_mean = (np.mean(shuffled_TEs[0, :]), \
               np.mean(shuffled_TEs[1, :]))

    ## Return the self.DF value to the unshuffled case
    return p_values, z_scores, TE_mean

def shuffle_along_axis(a, axis):
    idx = np.random.rand(*a.shape).argsort(axis=axis)
    return np.take_along_axis(a, idx, axis=axis)